In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import numpy as np
import gym
import matplotlib.pyplot as plt
import gym
import highway_env
import imageio
from IPython.display import Video

In [11]:
# define policy network
class policy_net(nn.Module):
  # nS: state space size, nH: n. of neurons in hidden layer, nA: size action space
    def __init__(self, nS, nH, nA): 
        super(policy_net, self).__init__()
        self.h = nn.Linear(nS, nH)
        self.out = nn.Linear(nH, nA)

    # define forward pass with one hidden layer with ReLU activation and sofmax after output layer
    def forward(self, x):
        x = F.relu(self.h(x))
        x = F.softmax(self.out(x), dim=1)
        return x

In [12]:
# create environment
env = gym.make("merge-v0")
# instantiate the policy
policy = policy_net(env.observation_space.shape[0]*5, 20, env.action_space.n)

C:\Users\Nikoo\AppData\Roaming\Python\Python38\site-packages\gym\utils\passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (5, 5)
  logger.warn(


In [13]:
policy.load_state_dict(torch.load('merge_weights.pth'))

<All keys matched successfully>

In [15]:
Terminal = False
images = []  
state, _ = env.reset()
img = env.render()
images.append(img)
while not Terminal:
    probs = policy(torch.tensor(state.flatten()).unsqueeze(0).float())
    # sample an action from that set of probs
    sampler = Categorical(probs)
    action = sampler.sample()
    state, reward, Terminal,_,_ = env.step(action.item())
    img = env.render()
    images.append(img)

imageio.mimsave('test.mp4', [np.array(img) for i, img in enumerate(images)], fps=1)

Video("test.mp4", embed = True)


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 150) to (608, 160) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
